In [1]:
import numpy as np
import matplotlib.pyplot as plt
import VisualDataMaker
import CNNRegressionAndSoftmax as CRS
import tensorflow as tf


In [2]:
data, OHE_LABELS = VisualDataMaker.combine("./random_BG_IMG/n_data/n_data/*.jpg", "./original_DOB_images/*.png")
bounded_DOB = VisualDataMaker.plot_bounding_box(data)

In [3]:
batch = []
regr_truth = []
class_truth = []
for i in range(len(data)):
    batch.append(data[i][0])
    regr_truth.append([data[i][2][0], data[i][2][1], data[i][3]])
    class_truth.append(OHE_LABELS[data[i][1]])
    

In [4]:
batch = np.array(batch)
regr_truth = np.array(regr_truth)
class_truth = np.array(class_truth)

In [5]:
batch.shape, regr_truth.shape, class_truth.shape

((19800, 224, 224, 3), (19800, 3), (19800, 10))

In [6]:
batch = tf.keras.backend.constant(batch)

In [7]:
model = CRS.getUntrainedModel()

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [9]:
#model.fit(batch, [class_truth, regr_truth], epochs=100, batch_size=64)

In [ ]:
#model.save("./Saved_Models/CNN_Regression_and_Softmax")

INFO:tensorflow:Assets written to: ./Saved_Models/assets


INFO:tensorflow:Assets written to: ./Saved_Models/assets
